<a href="https://colab.research.google.com/github/gaurinotgauri/Satellite-image-classifier/blob/main/Satellite_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

Sun Aug 18 13:07:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [4]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1


In [6]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [8]:
# re-size all the images
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/satellite images/train'
valid_path = '/content/drive/MyDrive/satellite images/valid'


In [9]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [10]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [11]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/satellite images/train/*')
folders

['/content/drive/MyDrive/satellite images/train/desert',
 '/content/drive/MyDrive/satellite images/train/cloudy',
 '/content/drive/MyDrive/satellite images/train/water',
 '/content/drive/MyDrive/satellite images/train/green_area']

In [12]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [14]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 111, 111, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 111, 111, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 109, 109, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 109, 109, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 109, 109, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 109, 109, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 109, 109, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 54, 54, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 54, 54, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 54, 54, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 52, 52, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 52, 52, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,007,588 (83.95 MB)

 Trainable params: 204,804 (800.02 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [35]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/satellite images/train',
                                                 target_size = (224, 224),
                                                 class_mode = 'categorical')

Found 4831 images belonging to 4 classes.


In [36]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/satellite images/valid',
                                            target_size = (224, 224),
                                            class_mode = 'categorical')


Found 800 images belonging to 4 classes.


In [37]:
# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  verbose=True
)

151/151 ━━━━━━━━━━━━━━━━━━━━ 79s 503ms/step - accuracy: 0.9765 - loss: 0.9362 - val_accuracy: 0.9812 - val_loss: 0.6877


In [39]:
from tensorflow.keras.models import load_model
model.save('model_inception.h5')

In [40]:
y_pred = model.predict(test_set)
y_pred

25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 142ms/step


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.4012985e-45, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.4742794e-28, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.9463416e-37, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [41]:
y_pred = np.argmax(y_pred, axis=1)

In [42]:
y_pred

array([3, 3, 0, 3, 2, 2, 3, 2, 1, 1, 2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 1, 1,
       3, 0, 0, 2, 2, 2, 1, 3, 1, 3, 3, 2, 1, 1, 2, 2, 2, 1, 0, 1, 3, 2,
       0, 0, 0, 0, 2, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 1, 1, 0, 0, 2, 0, 0,
       3, 0, 3, 3, 1, 1, 0, 1, 3, 3, 3, 0, 3, 1, 1, 0, 3, 2, 2, 2, 1, 3,
       0, 2, 1, 2, 0, 0, 2, 2, 2, 2, 3, 3, 3, 2, 2, 1, 2, 3, 2, 0, 1, 3,
       1, 1, 3, 2, 0, 3, 3, 3, 2, 3, 2, 1, 1, 0, 0, 0, 1, 3, 2, 2, 1, 2,
       2, 0, 0, 2, 1, 0, 3, 1, 0, 2, 2, 1, 1, 3, 3, 1, 1, 3, 2, 0, 1, 3,
       0, 2, 2, 0, 0, 2, 1, 3, 2, 1, 2, 2, 0, 1, 0, 0, 1, 1, 2, 2, 3, 0,
       0, 2, 0, 2, 1, 3, 3, 3, 2, 2, 2, 2, 0, 0, 3, 0, 2, 1, 0, 3, 3, 0,
       0, 1, 3, 1, 0, 0, 0, 2, 0, 2, 3, 2, 3, 1, 0, 0, 0, 3, 1, 1, 2, 1,
       1, 2, 0, 2, 3, 2, 1, 2, 2, 3, 2, 0, 0, 0, 2, 1, 1, 2, 0, 1, 3, 0,
       3, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 3, 1, 0, 0, 2, 3, 0, 1, 1, 0, 0,
       3, 0, 1, 0, 3, 1, 1, 1, 3, 0, 0, 0, 0, 2, 3, 1, 1, 3, 1, 0, 1, 1,
       0, 1, 3, 3, 0, 2, 0, 0, 0, 1, 1, 2, 3, 0, 1,